In [6]:
class Model:
    
    adds_revenue_model = ''
    start_year, start_month, end_year, end_month = 0, 0, 0, 0
    
    
    def __init__(self, adds_model, dict_dates):
        
        self.adds_revenue_model = adds_model
        self.start_year = int(dict_dates('StartYear'))
        self.start_month = int(dict_dates('StarthMonth'))
        self.end_year = int(dict_dates('EndYear'))
        self.end_month = int(dict_dates('EndMonth'))
    
    
    def __del__(self):
        pass
    
    
    def __users_this_month(self, users_previous_month, users_retention, new_users):
        
        users_this_month = users_previous_month * users_retention + new_users
        return users_this_month
    
    
    def __revenue_premium_account(self, users_this_month, percent_premium_accounts, premium_price):
        revenue_premium_account = users_this_month * percent_premium_accounts * premium_price
        return revenue_premium_account
    
    
    def __revenue_cpm(self, sessions_count, cpm, avg_session_min, add_impressions_min, fill_rate):
        
        revenue = sessions_count/1000 * avg_session_min * add_impressions_min * fill_rate
        return revenue
    
    
    def __revenue_cpc(self, sessions_count, cpc, ctr):
        
        renevue = sessions_count * cpc * ctr
        return renevue
    
    
    def __revenue_cpa(self, sessions_count, cpa, ctr, cvr):
        
        renevue = sessions_count * cpa * ctr * cvr
        return renevue
    
    
    def __adds_revenue(self, sessions_count, cost_parameter,\
                       avg_session_min=0, add_impressions_min=0, fill_rate=0,\
                       ctr=0, cvr=0):
        
        adds_revenue = 0
        
        if self.adds_revenue_model == 'CPM':
            adds_revenue = __revenue_cpm(sessions_count, cost_parameter, avg_session_min, add_impressions_min, fill_rate)
        
        elif self.adds_revenue_model == 'CPC':
            adds_revenue = __revenue_cpc(sessions_count, cost_parameter, ctr)
        
        elif self.adds_revenue_model == 'CPA':
            adds_revenue = __revenue_cpa(sessions_count, cost_parameter, ctr, cvr)
        
        return adds_revenue
    
    
    def __costs_this_month(self):
        pass
    
    
    def __revenue_this_month(self, revenue_adds, revenue_premium, costs):
        total_revenue = revenue_adds + revenue_premium - costs
        return total_revenue
    
    
    def __simulate_month(self):
        pass
    
    
    def simulate(self):
        pass
    
    